<a href="https://colab.research.google.com/github/BINDHUPALLE/music_reccomendor/blob/main/music_recom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
with open("spotify.csv", 'r') as f:
    lines = f.readlines()
    print(lines[4230:4240])  # Print around the problematic line


In [ ]:
df = pd.read_csv("spotify.csv", engine='python', dtype=str)
df.head()


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
#del duplicate
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

#text cleaning


In [ ]:
df['text'] = df['text'].str.lower()
df['text'] = df['text'].replace(r'\n', ' ', regex=True)
df['text'] = df['text'].replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # remove punctuation


In [ ]:
df.head()

In [ ]:
df['artist'] = df['artist'].str.lower().str.replace(' ', '')


In [ ]:
df.head()

In [ ]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:

CLIENT_ID = "70a9fb89662f4dac8d07321b259eaad7"
CLIENT_SECRET = "4d6710460d764fbbb8d8753dc094d131"

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_song_album_cover_url(song_name, artist_name):
    try:
        search_query = f"track:{song_name} artist:{artist_name}"
        results = sp.search(q=search_query, type="track")
        track = results["tracks"]["items"][0]
        album_cover_url = track["album"]["images"][0]["url"]
        spotify_url = track["external_urls"]["spotify"]
        return album_cover_url, spotify_url
    except:
        return None, None

In [ ]:
filtered_rows = []

for idx, row in df.sample(n=15000, random_state=42).iterrows():
    poster, url = get_song_album_cover_url(row['song'], row['artist'])
    if poster and url:
        row['album_cover_url'] = poster
        row['spotify_url'] = url
        filtered_rows.append(row)

# Create filtered DataFrame
filtered_music_df = pd.DataFrame(filtered_rows)

In [ ]:
filtered_music_df.head()

In [ ]:
filtered_music_df.to_csv("filtered_music.csv", index=False)


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df=pd.read_csv("filtered_music.csv")

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# STEP 1: Get unique artist embeddings
artist_embeddings = {}
for artist in df['artist'].unique():
    text = f"Artist: {artist}"
    vec = model.encode(text, convert_to_tensor=True)
    artist_embeddings[artist] = vec

# STEP 2: Compute combined embedding (lyrics + artist)
def get_combined_vector(row):
    lyric_vec = model.encode(row['text'], convert_to_tensor=True)
    artist_vec = artist_embeddings[row['artist']]
    combined_vec = (0.75*lyric_vec +0.25* artist_vec)  # or use concat for more complexity
    return combined_vec

In [ ]:
df['combined_vector'] = df.apply(get_combined_vector, axis=1)

# STEP 3: Convert all combined vectors into a matrix
embedding_matrix = torch.stack(df['combined_vector'].tolist()).cpu().numpy()

# STEP 4: Compute cosine similarity matrix
similarity_matrix = cosine_similarity(embedding_matrix)

In [ ]:
df.shape

In [ ]:
similarity_matrix.shape

In [ ]:
import gzip
import pickle
import numpy as np
from tqdm import tqdm  # Optional: progress bar

top_n = 30
similarity_dict = {}

for idx in tqdm(range(len(df))):
    song_name = df.iloc[idx]['song']
    # Get sorted indices of top-N similar songs (excluding itself)
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_similar = sim_scores[1:top_n+1]  # Exclude itself at index 0

    similarity_dict[song_name] = [
        {
            "song": df.iloc[i]['song'],
            "artist": df.iloc[i]['artist'],
            "score": float(score)
        }
        for i, score in top_similar
    ]

# Save this much smaller structure
with gzip.open("top30_similar.pkl.gz", "wb") as f:
    pickle.dump(similarity_dict, f)


In [ ]:

def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity_matrix[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [ ]:
df.tail()